In [ ]:
!pip install deap

In [ ]:
import math
import random
import pandas as pd

import numpy
from deap import base
from deap import creator
from deap import algorithms
from deap import tools
import matplotlib.pyplot as plt

In [ ]:
#Definir acesso ao Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def distL2(x1,y1, x2,y2):
    """Compute the L2-norm (Euclidean) distance between two points.

    The distance is rounded to the closest integer, for compatibility
    with the TSPLIB convention.

    The two points are located on coordinates (x1,y1) and (x2,y2),
    sent as parameters"""
    xdiff = x2 - x1
    ydiff = y2 - y1
    return int(math.sqrt(xdiff*xdiff + ydiff*ydiff) + .5)

In [ ]:
def distL1(x1,y1, x2,y2):
    """Compute the L1-norm (Manhattan) distance between two points.

    The distance is rounded to the closest integer, for compatibility
    with the TSPLIB convention.

    The two points are located on coordinates (x1,y1) and (x2,y2),
    sent as parameters"""
    return int(abs(x2-x1) + abs(y2-y1)+.5)

In [ ]:
def calc_matriz(coord, dist):
    """Calcula a matriz de distância entre as cidades.

    Utiliza a função armazena em 'dist' para calcular as distâncias
    entre dois pontos quaisquer.

    Parametross:
    -coord -- lista de tuplas com as coordenadas de todos os pontos, [(x1,y1),...,(xn,yn)]
    -dist -- funçaõ distância
    """
    n = len(coord)
    # dicionario com as com coordenadas (chave) e distâncias (valor)
    D = {}
    for i in range(n-1):
        for j in range(i+1,n):
            (x1,y1) = coord[i]
            (x2,y2) = coord[j]
            D[i,j] = dist(x1,y1,x2,y2)
            D[j,i] = D[i,j]
    return n,D

In [ ]:
def read_tsplib(filename):
    "basic function for reading a TSP problem on the TSPLIB format"
    "NOTE: only works for 2D euclidean or manhattan distances"
    f = open(filename, 'r');

    line = f.readline()
    while line.find("EDGE_WEIGHT_TYPE") == -1:
        line = f.readline()

    if line.find("EUC_2D") != -1:
        dist = distL2
    elif line.find("MAN_2D") != -1:
        dist = distL1
    else:
        print("cannot deal with non-euclidean or non-manhattan distances")
        raise Exception

    while line.find("NODE_COORD_SECTION") == -1:
        line = f.readline()

    xy_positions = []
    while 1:
        line = f.readline()
        if line.find("EOF") != -1: break
        (i,x,y) = line.split()
        # print(dist,i,x,y)
        x = float(x)
        y = float(y)
        xy_positions.append((x,y))

    n,D = calc_matriz(xy_positions, dist)
    return n, xy_positions, D

In [ ]:
#Função de custo, utilizada como fitness_function no DEAP
def custo(tour,D):
    """Calcula a distÇancia da rota de acordo com a matriz 'D'."""
    z = D[tour[-1], tour[0]]    # aresta a entre a primeira e última cidade da rota
    for i in range(1,len(tour)):
        z += D[tour[i], tour[i-1]] # atualiza o custo da rota a partir da cidade i-1 to i
    return z,

In [ ]:
#Gera uma lista de tamanho n cidades que serão visitadas, sendo n = número de cidades da base de dados
def randtour(n):
    """Gera uma solução aleatória de tamanho 'n'."""
    sol = list(range(n)) # Gera uma lista sequencial
    random.shuffle(sol) # Embaralha a lista
    return sol


In [ ]:
#Usada para pegar os dados (max,min,std) dos individuos da população
def statistics(individual):
  return individual.fitness.values

In [ ]:
#Recebe o nome do arquivo .tsp e retorna n (quantidade de cidades) e D (dicionário com coordenadas e distancias)
def ler_dataset(dataset):
  path = '/content/drive/MyDrive/Inteligência Artificial EC47A/Busca /Genética/Projeto 1/'+dataset
  return read_tsplib(path)

In [ ]:
# Cria o tipo de função fitness e indivíduo
creator.create("Minimization", base.Fitness, weights=(-1.0, ))
creator.create("Individual", list, fitness=creator.Minimization)

# Gera e retorna o objeto toolbox responsável por registrar as configurações do framewrok
def callToolBox(change):
  toolbox = base.Toolbox()


  # Registra os nomes e os tipos de individuo, fiteness e população
  toolbox.register("attr_bool", randtour, n)
  toolbox.register("gene", tools.initIterate, creator.Individual,
                  toolbox.attr_bool)
  toolbox.register("population", tools.initRepeat, list, toolbox.gene)

  # Registra os operadores. Deve-se manter os nomes evaluate, mate, mutate e select
  toolbox.register("evaluate", custo, D=D)
  toolbox.register("mutate", tools.mutShuffleIndexes, indpb = 0.01)
  t_size = 6


  if(change == "PM+T"):
    toolbox.register("mate", tools.cxPartialyMatched)
    toolbox.register("select", tools.selTournament, tournsize = t_size)
  elif(change=="PM+R"):
    toolbox.register("mate", tools.cxPartialyMatched)
    toolbox.register("select", tools.selRoulette)
  elif(change == "UPM+T"):
    toolbox.register("mate", tools.cxUniformPartialyMatched, indpb = 0.1)
    toolbox.register("select", tools.selTournament, tournsize = t_size)
  elif(change == "UPM+R"):
    toolbox.register("mate", tools.cxUniformPartialyMatched, indpb = 0.1)
    toolbox.register("select", tools.selRoulette)
  elif(change == "O+T"):
    toolbox.register("mate", tools.cxOrdered)
    toolbox.register("select", tools.selTournament, tournsize = t_size)
  elif(change == "O+R"):
    toolbox.register("mate", tools.cxOrdered)
    toolbox.register("select", tools.selRoulette)
  return toolbox


# **SALVAMENTO DOS DADOS**

In [ ]:
def gerar_df():
  #Criação das colunas para exportação da planilha
  colunas = {'Geração':[],'Melhor Solução':[],'Média':[],'Desvio':[]}
  df = pd.DataFrame(colunas)

  #Laço de repetição para testagem das 6 combinações
  for i in range(0,6):
    toolbox = callToolBox(tipo[i])
    df.loc[len(df.index)] = ['Combinação ' + str(comb[i]), '', '', ''] #Adicionando linha à planilha


    #Laço de repetição para testar 10 vezes cada combinação
    for i in range(0,10):
      populacao = toolbox.population(n = 100)

      prob_cross = 40.0
      prob_mut = 0.20
      geracoes = 500

      # stats = tools.Statistics(key=lambda Individual: Individual.fitness.values)
      stats = tools.Statistics(statistics)
      stats.register("max", numpy.max)
      stats.register("min", numpy.min)
      stats.register("med", numpy.mean)
      stats.register("std", numpy.std)

      populacao, info = algorithms.eaSimple(populacao, toolbox, prob_cross,
                                                prob_mut,
                                                geracoes, stats,verbose=False)

      melhor = info[geracoes]
      ms = melhor.get('min')
      media = melhor.get('med')
      desvio = melhor.get('std')

      df.loc[len(df.index)] = [str(i+1), ms, media, desvio] #Adicionando linha à planilha
  return df




In [ ]:
#Organizando combinações dos operadores
comb1 = "cxPartialyMatched # mutShuffleIndexes # selTournament"
comb2 = "cxPartialyMatched # mutShuffleIndexes # selRoulette"

comb3= "cxUniformPartialyMatched # mutShuffleIndexes # selTournament"
comb4 = "cxUniformPartialyMatched # mutShuffleIndexes # selRoulette"

comb5 = "cxOrdered # mutShuffleIndexes # selTournament"
comb6 = "cxOrdered # mutShuffleIndexes # selRoulette"

comb = [comb1,comb2,comb3,comb4,comb5,comb6,]

#Definindo os tipos para mudança do toolbox
tipo = ["PM+T", "PM+R", "UPM+T", "UPM+R", "O+T", "O+R"]

#Definindo os arquivo de leitura e salvamento
entrada = ['bier127.tsp', 'kroA100.tsp', 'kroC100.tsp' , 'ch130.tsp', 'berlin52.tsp']
saida = ['bier127.xlsx', 'kroA100.xlsx', 'kroC100.xlsx' , 'ch130.xlsx', 'berlin52.xlsx']

# **Resultados**

In [ ]:
for dataset in range(0,5):
  n, coord, D = ler_dataset(entrada[dataset])
  df = gerar_df()
  display(df)
  #df.to_excel(saida[dataset])
